In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/county_2018.csv')
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,county,office,district,stage,special,rank,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
0,2018,Alabama,AL,1,63,41,Autauga,"Associate Justice of the Supreme Court, Place 1",statewide,gen,False,NaN,NaN,NaN,True,absentee,7.0,1705793.0,False,20190430
1,2018,Alabama,AL,1,63,41,Autauga,"Associate Justice of the Supreme Court, Place 1",statewide,gen,False,NaN,NaN,NaN,True,election day,308.0,1705793.0,False,20190430
2,2018,Alabama,AL,1,63,41,Autauga,"Associate Justice of the Supreme Court, Place 1",statewide,gen,False,NaN,NaN,NaN,True,provisional,0.0,1705793.0,False,20190430
3,2018,Alabama,AL,1,63,41,Autauga,"Associate Justice of the Supreme Court, Place 1",statewide,gen,False,NaN,Over Votes,NaN,False,absentee,0.0,1705793.0,False,20190430
4,2018,Alabama,AL,1,63,41,Autauga,"Associate Justice of the Supreme Court, Place 1",statewide,gen,False,NaN,Over Votes,NaN,False,election day,0.0,1705793.0,False,20190430


In [3]:
df2 = df[df['district']=='statewide']
df2 = df2[df2['office'] == 'US Senator']
df3 = df2.dropna(subset=['party'])
writein = df2[df2['writein'] & (df2['party'].isna())]

In [4]:
df4 = pd.concat([df3, writein])

In [5]:
df5 = df4.sort_values(by=['state','county'])

In [6]:
df6 = df5[['year','state','state_po','county','office','district','stage','special','candidate','party','writein','mode','candidatevotes','totalvotes']]

In [7]:
df7 = df6.groupby(['year','state','state_po', 'county','office','district','stage','special', 'candidate','party','writein'], dropna=False)['candidatevotes'].sum().reset_index()

df7.head()

,year,state,state_po,county,office,district,stage,special,candidate,party,writein,candidatevotes
0,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,Angela Green,green,False,961.0
1,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,Kyrsten Sinema,democrat,False,16298.0
2,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,Martha McSally,republican,False,7810.0
3,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,NaN,NaN,True,18.0
4,2018,Arizona,AZ,Cochise,US Senator,statewide,gen,False,Angela Green,green,False,1212.0


In [8]:
def remove_newline(s):
    if type(s)==float:
        return s
    return s.replace('\n', ' ')

In [9]:
df8 = df7.copy()
df8['candidate'] = df7['candidate'].apply(remove_newline)

In [10]:
mn_special = ['Tina Smith', 'Karin Housley', 'Sarah Wellington', 'Jerry Trooien']
mn_reg = ['Amy Klobuchar', 'Jim Newberger', 'Dennis Schuller', 'Paula M Overby']

ms_special = ['Cindy Hyde-Smith', 'Mike Epsy', 'Chris McDaniel', 'Tobey Bernard Bartee']
ms_reg = ['Roger F. Wicker', 'David Baria', 'Danny Bedwell', "Shawn O\'Hara"]

In [11]:
for index, row in df8.iterrows():
    if row['state_po'] in ['MN', 'MS']:
        if (row['candidate'] in mn_special) or (row['candidate'] in ms_special):
            df8.at[index, 'special'] = True
            pass

In [12]:
#df8.to_csv('cleandata.csv',index=False)

# FURTHER CLEANING

In [13]:
#df = pd.read_csv('cleandata.csv')
df = df8.copy()

In [14]:
data16 = pd.read_csv('data/2016_US_County_Level_Presidential_Results.csv', index_col=0)

In [15]:
states = list(df['state_po'].unique())

In [16]:
d16 = data16[data16['state_abbr'].isin(states)]
d16.head()

,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
171,12196.0,5315.0,18659.0,0.653626,0.284849,"6,881",36.88%,AZ,Apache County,4001
172,15291.0,25036.0,43147.0,0.354393,0.580249,"9,745",22.59%,AZ,Cochise County,4003
173,25308.0,16573.0,44929.0,0.563289,0.368871,"8,735",19.44%,AZ,Coconino County,4005
174,6746.0,13672.0,21398.0,0.315263,0.638938,"6,926",32.37%,AZ,Gila County,4007
175,3301.0,8025.0,11939.0,0.276489,0.672167,"4,724",39.57%,AZ,Graham County,4009


### MOV column is D-R

In [17]:
d16_c = d16.copy()
d16_c['MOV16'] = d16['per_dem'] - d16['per_gop']
d16_c.head()

,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips,MOV16
171,12196.0,5315.0,18659.0,0.653626,0.284849,"6,881",36.88%,AZ,Apache County,4001,0.368776
172,15291.0,25036.0,43147.0,0.354393,0.580249,"9,745",22.59%,AZ,Cochise County,4003,-0.225856
173,25308.0,16573.0,44929.0,0.563289,0.368871,"8,735",19.44%,AZ,Coconino County,4005,0.194418
174,6746.0,13672.0,21398.0,0.315263,0.638938,"6,926",32.37%,AZ,Gila County,4007,-0.323675
175,3301.0,8025.0,11939.0,0.276489,0.672167,"4,724",39.57%,AZ,Graham County,4009,-0.395678


In [18]:
import re

def remove_punctuation(text):
    # Define the pattern to match punctuation
    pattern = r'[^\w\s]'

    # Remove punctuation using regex
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

In [19]:
d16_c['county_str'] = d16_c['county_name'].str.lower().str.replace(' ','').str.replace('county','').apply(remove_punctuation)
d16_c['county_str'] = d16_c['county_str'] + '_' + d16_c['state_abbr'].str.lower()
d16_c.head()

,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips,MOV16,county_str
171,12196.0,5315.0,18659.0,0.653626,0.284849,"6,881",36.88%,AZ,Apache County,4001,0.368776,apache_az
172,15291.0,25036.0,43147.0,0.354393,0.580249,"9,745",22.59%,AZ,Cochise County,4003,-0.225856,cochise_az
173,25308.0,16573.0,44929.0,0.563289,0.368871,"8,735",19.44%,AZ,Coconino County,4005,0.194418,coconino_az
174,6746.0,13672.0,21398.0,0.315263,0.638938,"6,926",32.37%,AZ,Gila County,4007,-0.323675,gila_az
175,3301.0,8025.0,11939.0,0.276489,0.672167,"4,724",39.57%,AZ,Graham County,4009,-0.395678,graham_az


In [20]:
df_c = df.copy()

df_c['county_str'] = df['county'].str.lower().str.replace(' ','').str.replace('county','').apply(remove_punctuation)
df_c['county_str'] = df_c['county_str'] + '_' + df_c['state_po'].str.lower()

df_c

,year,state,state_po,county,office,district,stage,special,candidate,party,writein,candidatevotes,county_str
0,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,Angela Green,green,False,961.0,apache_az
1,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,Kyrsten Sinema,democrat,False,16298.0,apache_az
2,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,Martha McSally,republican,False,7810.0,apache_az
3,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,NaN,NaN,True,18.0,apache_az
4,2018,Arizona,AZ,Cochise,US Senator,statewide,gen,False,Angela Green,green,False,1212.0,cochise_az
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9164,2018,Wyoming,WY,Washakie,US Senator,statewide,gen,False,Write-Ins,NaN,True,6.0,washakie_wy
9165,2018,Wyoming,WY,Weston,US Senator,statewide,gen,False,Gary Trauner,democrat,False,321.0,weston_wy
9166,2018,Wyoming,WY,Weston,US Senator,statewide,gen,False,John Barrasso,republican,False,2275.0,weston_wy
9167,2018,Wyoming,WY,Weston,US Senator,statewide,gen,False,Joseph Porambo,libertarian,False,89.0,weston_wy


In [21]:
df_c = df_c[df_c['special']==False]

In [22]:
won_col = []

county_max = df_c.groupby('county_str')['candidatevotes'].max().to_dict()

for index, row in df_c.iterrows():
    #display(row)
    if row['candidatevotes'] == county_max[row['county_str']]:
        won_col.append(True)
    else:
        won_col.append(False)
        
df_c['won'] = pd.Series(won_col)

<ipython-input-22-946173575369>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c['won'] = pd.Series(won_col)


In [23]:
county_total = df_c.groupby('county_str')['candidatevotes'].sum().to_dict()

In [24]:
def convert(county):
    return county_total[county]

In [25]:
#df_c['total_votes'] = county_total[df_c['county_str'].astype('str')]
df_c['total'] = df_c['county_str'].apply(convert)

<ipython-input-25-a7a91d677f8b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c['total'] = df_c['county_str'].apply(convert)


In [26]:
def party_uniform(party):
    if party == 'democratic-farmer-labor' or party == 'democratic-npl':
        return 'democrat'
    else:
        return party


df_c['party'] = df_c['party'].apply(party_uniform)

<ipython-input-26-26ee53351fb7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c['party'] = df_c['party'].apply(party_uniform)


In [27]:
MOV18 = dict()

for county in list(set(df_c['county_str'])):
    frame = df_c[(df_c['county_str'] == county) & (df_c['special']==False)]
    
    for index, row in frame.iterrows():
        #if row['county_str'] == 'ramsey_mn':
        #    print(row)
        
        
        if row['state_po'] == 'CA':
            dem_pct = 1.0
            gop_pct = 0.0
        elif (row['state_po'] == 'VT') or (row['state_po'] == 'ME'):
            if (row['party'] == 'independent') and ((row['candidate'] == 'Bernie Sanders') or (row['candidate'] == 'Angus S. King Jr.')):
                dem_pct = row['candidatevotes'] / row['total']
            elif row['party'] == 'republican':
                gop_pct = row['candidatevotes'] / row['total']
        else:
            if row['party'] == 'democrat':
                dem_pct = row['candidatevotes'] / row['total']
            elif row['party'] == 'republican':
                gop_pct = row['candidatevotes'] / row['total']
            
    mov = dem_pct - gop_pct
    MOV18[county] = mov
        

In [28]:
def mov18(county):
    return MOV18[county]

In [29]:
df_c['MOV18'] = df_c['county_str'].apply(mov18)
df_c

<ipython-input-29-5bae0d3f4f5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c['MOV18'] = df_c['county_str'].apply(mov18)


,year,state,state_po,county,office,district,stage,special,candidate,party,writein,candidatevotes,county_str,won,total,MOV18
0,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,Angela Green,green,False,961.0,apache_az,False,25087.0,0.338343
1,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,Kyrsten Sinema,democrat,False,16298.0,apache_az,True,25087.0,0.338343
2,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,Martha McSally,republican,False,7810.0,apache_az,False,25087.0,0.338343
3,2018,Arizona,AZ,Apache,US Senator,statewide,gen,False,NaN,NaN,True,18.0,apache_az,False,25087.0,0.338343
4,2018,Arizona,AZ,Cochise,US Senator,statewide,gen,False,Angela Green,green,False,1212.0,cochise_az,False,45549.0,-0.209577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9164,2018,Wyoming,WY,Washakie,US Senator,statewide,gen,False,Write-Ins,NaN,True,6.0,washakie_wy,NaN,3110.0,-0.590032
9165,2018,Wyoming,WY,Weston,US Senator,statewide,gen,False,Gary Trauner,democrat,False,321.0,weston_wy,NaN,2688.0,-0.726935
9166,2018,Wyoming,WY,Weston,US Senator,statewide,gen,False,John Barrasso,republican,False,2275.0,weston_wy,NaN,2688.0,-0.726935
9167,2018,Wyoming,WY,Weston,US Senator,statewide,gen,False,Joseph Porambo,libertarian,False,89.0,weston_wy,NaN,2688.0,-0.726935


In [30]:
merged = df_c.merge(d16_c, on='county_str')

In [31]:
display_cols = ['state', 'state_po', 'county_str','office', 'special', 'candidate', 'party', 'writein', \
                'candidatevotes', 'won', 'MOV16', 'MOV18']

merged_final = merged[display_cols]

In [32]:
merged_final['16to18swing'] = merged_final['MOV18'] - merged_final['MOV16']

<ipython-input-32-f9686cd6921d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_final['16to18swing'] = merged_final['MOV18'] - merged_final['MOV16']


In [33]:
merged_final.to_csv('updatedcleandata.csv',index=False)